In [12]:
# pip install ultralytics opencv-python
import cv2, time
from ultralytics import YOLO

import os
os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = ";".join([
    "rtsp_transport;udp",   # UDP
    "fflags;nobuffer",      # demuxer bufferını en aza indir
    "max_delay;0",          # bekleme yok
    "reorder_queue_size;0", # paket sıralama kuyruğu kapalı
    "analyzeduration;0",    # analiz süresi çok kısa
    "probesize;32"          # prob boyutu küçük
])

MODEL = r"C:\Users\524ha\Desktop\Araştırmalarım\data_managment\pt\epoch480.pt"  # .pt veya .engine
SOURCE = r"C:\Users\524ha\Desktop\Araştırmalarım\data_managment\test.mp4"  # webcam için 0 yaz
INFER_SIZE = 1280           # çıkarım boyutu (gerekirse 960/1664)
DISPLAY_SCALE = 0.6         # pencereye küçük göstermek için ölçek (0.5–0.7 iyi)

model = YOLO(MODEL)
cap = cv2.VideoCapture(SOURCE)
if not cap.isOpened():
    raise RuntimeError(f"Video açılamadı: {SOURCE}")

fps = 0.0
alpha = 0.9  # FPS yumuşatma

while True:
    ok, frame = cap.read()
    if not ok:
        break

    t0 = time.perf_counter()
    res = model.predict(
        frame, imgsz=INFER_SIZE, device=0, half=True,
        conf=0.4, iou=0.30, max_det=1000, verbose=False
    )
    view = res[0].plot()  # çizimli kare
    dt = time.perf_counter() - t0
    fps = alpha*fps + (1-alpha)*(1.0/dt)

    # Küçük gösterim
    view_small = cv2.resize(view, None, fx=DISPLAY_SCALE, fy=DISPLAY_SCALE,
                            interpolation=cv2.INTER_AREA)

    # FPS yaz
    # cv2.putText(view, f"FPS: {fps:.1f}", (12, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2, cv2.LINE_AA)

    cv2.imshow("YOLO (preview)", view)
    if cv2.waitKey(1) & 0xFF in (27, ord('q')):  # ESC veya q ile çık
        break

cap.release()
cv2.destroyAllWindows()


In [13]:
# pip install ultralytics opencv-python
import cv2, time
from ultralytics import YOLO
import os

os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = ";".join([
    "rtsp_transport;udp",   # UDP
    "fflags;nobuffer",      # demuxer bufferını en aza indir
    "max_delay;0",          # bekleme yok
    "reorder_queue_size;0", # paket sıralama kuyruğu kapalı
    "analyzeduration;0",    # analiz süresi çok kısa
    "probesize;32"          # prob boyutu küçük
])

MODEL = r"C:\Users\524ha\Desktop\Araştırmalarım\data_managment\pt\epoch480.pt"
SOURCE = r"C:\Users\524ha\Desktop\Araştırmalarım\data_managment\test.mp4"
INFER_SIZE = 1280
DISPLAY_SCALE = 0.6

model = YOLO(MODEL)
cap = cv2.VideoCapture(SOURCE)
if not cap.isOpened():
    raise RuntimeError(f"Video açılamadı: {SOURCE}")

# Çıkış videosu ayarları
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # .mp4 için codec
fps_out = int(cap.get(cv2.CAP_PROP_FPS)) or 25  # orijinal FPS veya 25
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter("output.mp4", fourcc, fps_out, (w, h))

fps = 0.0
alpha = 0.9

while True:
    ok, frame = cap.read()
    if not ok:
        break

    t0 = time.perf_counter()
    res = model.predict(
        frame, imgsz=INFER_SIZE, device=0, half=True,
        conf=0.4, iou=0.30, max_det=1000, verbose=False
    )
    view = res[0].plot()
    dt = time.perf_counter() - t0
    fps = alpha*fps + (1-alpha)*(1.0/dt)

    # FPS yazdır (kaydedilen videoya da geçsin)
    cv2.putText(view, f"FPS: {fps:.1f}", (12, 28),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2, cv2.LINE_AA)

    # Videoya kaydet
    out.write(view)

    # Küçük gösterim
    view_small = cv2.resize(view, None, fx=DISPLAY_SCALE, fy=DISPLAY_SCALE,
                            interpolation=cv2.INTER_AREA)
    cv2.imshow("YOLO (preview)", view_small)

    if cv2.waitKey(1) & 0xFF in (27, ord('q')):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


In [19]:
# pip install ultralytics opencv-python supervision
import os, cv2, time
import numpy as np
from ultralytics import YOLO
import supervision as sv

# ---- (Gerekirse RTSP/FFmpeg) ----
os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = ";".join([
    "rtsp_transport;udp",
    "fflags;nobuffer",
    "max_delay;0",
    "reorder_queue_size;0",
    "analyzeduration;0",
    "probesize;32"
])

# ---- AYARLAR ----
MODEL = r"C:\Users\524ha\Desktop\Araştırmalarım\data_managment\pt\palletson_j.pt"
SOURCE = r"C:\Users\524ha\Desktop\Araştırmalarım\data_managment\test.mp4"   # kamera için 0
INFER_SIZE = 1280
CONF_THR = 0.4
IOU_THR = 0.30
DEVICE = 0           # CUDA: 0, CPU: 'cpu'
HALF = True
DISPLAY_SCALE = 0.6
OUTPUT_PATH = "output_counted.mp4"

# Yalnızca belirli sınıfları saymak istersen liste ver (örn. [0]); tüm sınıflar için None
CLASS_FILTER = None  # örn. sadece palet sınıfı 0 ise: [0]

# Çizgi ayarları (yön fark etmeksizin ilk geçişte say)
LINE_ORIENTATION = "horizontal"  # "horizontal" (yatay) veya "vertical" (dikey)
LINE_Y_RATIO = 0.5               # yatay çizgi için 0.5 = ekran ortası
LINE_X_RATIO = 0.5               # dikey çizgi için 0.5 = ekran ortası

# ---- MODEL & VIDEO ----
model = YOLO(MODEL)
cap = cv2.VideoCapture(SOURCE)
if not cap.isOpened():
    raise RuntimeError(f"Video açılamadı: {SOURCE}")

W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
src_fps = cap.get(cv2.CAP_PROP_FPS)
fps_out = src_fps if src_fps and not np.isnan(src_fps) and src_fps > 0 else 25

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps_out, (W, H))

# ---- TAKİP & ÇİZİM ----
tracker = sv.ByteTrack()
box_annotator = sv.BoxAnnotator(thickness=2)
label_annotator = sv.LabelAnnotator(text_position=sv.Position.TOP_LEFT)

# Sayaç ve durumlar
count_total = 0
counted_ids = set()   # çizgiyi çoktan geçmiş ID'ler
prev_pos = {}         # track_id -> önceki (cx, cy)

# Çizgi konumu
line_color = (0, 255, 255)
line_thickness = 2
line = None
if LINE_ORIENTATION == "horizontal":
    line_y = int(H * LINE_Y_RATIO)
    line = ("h", line_y)
else:
    line_x = int(W * LINE_X_RATIO)
    line = ("v", line_x)

alpha, fps = 0.9, 0.0

while True:
    ok, frame = cap.read()
    if not ok:
        break

    t0 = time.perf_counter()
    res = model.predict(
        frame, imgsz=INFER_SIZE, device=DEVICE, half=HALF,
        conf=CONF_THR, iou=IOU_THR, max_det=1000, verbose=False
    )[0]

    dets = sv.Detections.from_ultralytics(res)

    # Sınıf filtresi (isteğe bağlı)
    if CLASS_FILTER is not None and len(dets) > 0:
        mask = np.isin(dets.class_id, CLASS_FILTER)
        dets = dets[mask]

    tracked = tracker.update_with_detections(dets)

    # Geçiş algılama (yönsüz, ilk geçişte say)
    for (x1, y1, x2, y2), tid in zip(tracked.xyxy, tracked.tracker_id):
        if tid is None:
            continue

        cx = int((x1 + x2) / 2)
        cy = int((y1 + y2) / 2)

        if tid in counted_ids:
            prev_pos[tid] = (cx, cy)
            continue

        p = prev_pos.get(tid, None)
        if p is not None:
            px, py = p
            if line[0] == "h":
                ly = line[1]
                # önceki ve şimdiki nokta çizginin karşı tarafında mı?
                if (py - ly) * (cy - ly) < 0:
                    count_total += 1
                    counted_ids.add(tid)
            else:
                lx = line[1]
                if (px - lx) * (cx - lx) < 0:
                    count_total += 1
                    counted_ids.add(tid)

        prev_pos[tid] = (cx, cy)

    # Görselleştirme
    labels = []
    for conf, cls, tid in zip(tracked.confidence, tracked.class_id, tracked.tracker_id):
        lab = f"ID {tid}" if tid is not None else "ID -"
        if cls is not None:
            lab += f" | cls {int(cls)}"
        if conf is not None:
            lab += f" | {conf:.2f}"
        labels.append(lab)

    annotated = box_annotator.annotate(scene=frame.copy(), detections=tracked)
    annotated = label_annotator.annotate(scene=annotated, detections=tracked, labels=labels)

    # Çizgiyi çiz
    if line[0] == "h":
        ly = line[1]
        cv2.line(annotated, (0, ly), (W - 1, ly), line_color, line_thickness)
    else:
        lx = line[1]
        cv2.line(annotated, (lx, 0), (lx, H - 1), line_color, line_thickness)

    # Sayaç kutusu
    cv2.rectangle(annotated, (10, 10), (280, 70), (0, 0, 0), -1)
    cv2.putText(annotated, f"Count: {count_total}", (20, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 0.95, (0, 255, 0), 2, cv2.LINE_AA)

    # FPS (opsiyonel)
    dt = time.perf_counter() - t0
    fps = alpha * fps + (1 - alpha) * (1.0 / dt)
    # cv2.putText(annotated, f"FPS: {fps:.1f}", (W - 160, 30),
    #             cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

    # Kaydet & göster
    out.write(annotated)
    preview = cv2.resize(annotated, None, fx=DISPLAY_SCALE, fy=DISPLAY_SCALE, interpolation=cv2.INTER_AREA)
    cv2.imshow("Drone Count (yönsüz çizgi geçişi)", preview)
    if cv2.waitKey(1) & 0xFF in (27, ord('q')):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Kaydedildi: {OUTPUT_PATH}")


Kaydedildi: output_counted.mp4


In [1]:
import cv2
cap = cv2.VideoCapture(0)

cap.release()